In [1]:
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM, TaskType, PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments, logging, pipeline
from torch import cuda, bfloat16
import transformers
from trl import DataCollatorForCompletionOnlyLM, SFTConfig, SFTTrainer
import pandas as pd
from textwrap import dedent
from datasets import Dataset, load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score
import warnings
from metrics import  calculate_metrics, calculate_metrics2, calc_mets_my
import gc
import sys
import time


In [2]:
PROJECT = "Llama3-8B-QLora-FineTune-Omni"
MODEL_NAME = 'meta-llama/Meta-Llama-3-8B-Instruct'

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


model_config = transformers.AutoConfig.from_pretrained(
    MODEL_NAME,
    token=True
)

In [4]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)


In [17]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for  param in model.parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [15]:
model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
        config=model_config,
        quantization_config=bnb_config,
        device_map='auto',
        token=True
    )
print("loaded model........")




Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

loaded model........


In [18]:
print_trainable_parameters(model)

trainable params: 1050939392 || all params: 4540600320 || trainable%: 23.145384264959926


In [22]:
num_parameters = sum(p.numel() for p in model.parameters())
print(num_parameters)

4708372480


In [20]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [21]:
print_trainable_parameters(model)

trainable params: 167772160 || all params: 4708372480 || trainable%: 3.5632728870252848


In [13]:
# del model
gc.collect()
torch.cuda.empty_cache()

In [5]:
def format_test(row: dict, scenario='fine-tune'):
    prompt = dedent(
        f"""
        Place 1: '{row["e1"]}'
        Place 2: '{row["e2"]}'
        
    """
    )
    messages = [
        {
            "role": "system",
            "content": "Do the two place descriptions refer to the same real-world place? Answer with 'Yes' if they do and 'No' if they do not.",
        },
        {"role": "user", "content": prompt},
    ]
    if scenario=="zero":
        full_prompt = messages[0]["content"] + prompt + "Answer: "
        return full_prompt
    else:
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [6]:
def format_test_distance(row, scenario="fine-tune"):
    prompt = dedent(
        f"""
        Place1: '{row["e1"]}'
        Place2: '{row["e2"]}'
        Distance: {row['distance']}
    
    """
    )
    messages = [
        {
            "role": "system",
            "content":  "Two place descriptions and the geographic distance between them is provided. Do the two place descriptions refer to the same real-world place? Answer with 'Yes' if they do and 'No' if they do not.",
        },
        {"role": "user", "content": prompt},
    ]
    if scenario=="zero":
        full_prompt = messages[0]["content"] + prompt + "Answer:"
        return full_prompt
    else:
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [7]:
def format_test_gtminer(row: dict, scenario):
    prompt = dedent(
        f"""
    Place 1: '{row["e1"]}'
    Place 2: '{row["e2"]}'
    
    """
    )
    messages = [
        {
            "role": "system",
            "content": "Two place descriptions are provided. Answer with 'same_as' if the first place is the same as the second place. Answer with 'part_of' if the first place is a part of the second place and is located inside the second place. Answer with 'serves' if the first place provides a service to the second place in terms of human mobility, assistance, etc. Answer with 'unknown' if the two places show none of these relations.",
        },
        {"role": "user", "content": prompt},
    ]
    
    if scenario=="zero":
        full_prompt = messages[0]["content"] + prompt + "Answer: "
        return full_prompt
    else:
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [8]:
def format_test_gtminer_simple(row: dict, scenario):
    prompt = dedent(
        f"""
    Place 1: '{row["e1"]}'
    Place 2: '{row["e2"]}'
    
    """
    )
    messages = [
        {
            "role": "system",
            "content": "Two place descriptions are provided. Predict the relation between them. Answer only with ‘same_as’, ‘part_of’, ‘serves’ or ‘unknown’",
        },
        {"role": "user", "content": prompt},
    ]
    
    if scenario=="zero":
        full_prompt = messages[0]["content"] + prompt + "Answer:"
        return full_prompt
    else:
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [9]:
def format_test_gtminer_distance(row: dict, scenario="fine-tune"):
    prompt = dedent(
        f"""
    Place 1: '{row["e1"]}'
    Place 2: '{row["e2"]}'
    Distance: {row["distance"]}
    Answer only with: same_as, part_of, serves, unknown
    
    """
    )
    messages = [
        {
            "role": "system",
            "content": "Two place descriptions and the geographic distance between them is provided. Answer with 'same_as' if the first place is the same as the second place. Answer with 'part_of' if the first place is a part of the second place and is located inside the second place. Answer with 'serves' if the first place provides a service to the second place in terms of human mobility, assistance, etc. Answer with 'unknown' if the two places show none of these relations.",
        },
        {"role": "user", "content": prompt},
    ]
    
    if scenario=="zero":
        full_prompt = messages[0]["content"] + prompt + "Answer: "
        return full_prompt
    else:
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [10]:
def format_test_gtminer2(row: dict, scenario="fine-tune"):
    prompt = dedent(
        f"""
        Place 1: '{row["e1"]}'
        Place 2: '{row["e2"]}'
        Answer only with: same_as, part_of, serves, unknown
    """
    )
    messages = [
        {
            "role": "system",
            "content": "Two place descriptions are provided. Answer with 'same_as' if the first place is the same as the second place. Answer with 'part_of' if the first place is a part of the second place and is located inside the second place. Answer with 'serves' if the first place provides a service to the second place in terms of human mobility, assistance, etc. Answer with 'unknown' if the two places show none of these relations.",
        },
        {"role": "user", "content": prompt},
    ]
    if scenario=="zero":
        full_prompt = messages[0]["content"] + prompt + "Answer: "
        return full_prompt
    else:
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [11]:
def format_test_gtminer3(row: dict, scenario="fine-tune"):
    prompt = dedent(
        f"""
    Place 1: '{row["e1"]}'
    Place 2: '{row["e2"]}'
    Answer only with: same-as, part-of, serves, unknown
    """
    )
    messages = [
        {
            "role": "system",
            "content": "Two place descriptions are provided. Predict the relation between the two places.",
        },
        {"role": "user", "content": prompt},
    ]
    
    if scenario=="zero":
        full_prompt = messages[0]["content"] + prompt + "Answer: "
        return full_prompt
    else:
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [12]:
dataset_folder_path = ['datasets\\my_data_plm\\auck\\', 'datasets\\my_data_plm\\hope\\', 'datasets\\my_data_plm\\norse\\','datasets\\my_data_plm\\north\\', 'datasets\\my_data_plm\\palm\\', 'datasets\\osm_fsq_plm\\edi\\', 'datasets\\osm_fsq_plm\\pit\\', 'datasets\\osm_fsq_plm\\sin\\', 'datasets\\osm_fsq_plm\\tor\\', 'datasets\\osm_yelp_plm\\edi\\', 'datasets\\osm_yelp_plm\\pit\\', 'datasets\\osm_yelp_plm\\sin\\', 'datasets\\osm_yelp_plm\\tor\\', 'datasets\\acheson_plm\\swiss\\']

In [17]:
logging.set_verbosity_error()
for dataset_folder in dataset_folder_path:
 
    warnings.filterwarnings("ignore")
    
    print(dataset_folder.split("\\"))
    dataset_output_path_1, dataset_output_path_2 = dataset_folder.split("\\")[-3], dataset_folder.split("\\")[-2]
        
    dataset = load_dataset(
        "json",
        data_files={"train": dataset_folder+"train.json", "valid": dataset_folder+"valid.json", "test": dataset_folder+"test.json"},
    )
    print("successfully loaded dataset.......")
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
        config=model_config,
        quantization_config=bnb_config,
        device_map='auto',
        token=True
    )
    
    print("loaded model........")
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        MODEL_NAME,
        token=True
    )
    
    print("loaded tokenizer........")
    PAD_TOKEN = tokenizer.eos_token
    tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
    tokenizer.padding_side = "right"
    
    
    # model = prepare_model_for_kbit_training(model)
    # model = get_peft_model(model, lora_config)
    
    print(dataset['train'][0]['text'])
    
    
    
    # OUTPUT_DIR = "experiments\\"+ dataset_output_path_1 +"\\"+ dataset_output_path_2+"\\"
    # 
    # os.makedirs(os.path.dirname(OUTPUT_DIR), exist_ok=True)
    # 
    # sft_config = SFTConfig(
    #     output_dir=OUTPUT_DIR,
    #     dataset_text_field="text",
    #     max_seq_length=256,
    #     num_train_epochs=2,
    #     per_device_train_batch_size=6,
    #     per_device_eval_batch_size=4,
    #     gradient_accumulation_steps=4,
    #     # optim="paged_adamw_8bit",
    #     # eval_strategy="steps",
    #     # eval_steps=0,
    #     #save_steps=0.2,
    #     logging_steps=10,
    #     learning_rate=1e-4,
    #     # fp16=True,  # or 
    #     bf16=True,
    #     save_strategy="steps",
    #     warmup_ratio=0.1,
    #     save_total_limit=0,
    #     lr_scheduler_type="constant",
    #     # report_to="tensorboard",
    #     save_safetensors=True,
    #     dataset_kwargs={
    #         "add_special_tokens": False,  
    #         "append_concat_token": False, 
    #     },
    # )
    # 
    # trainer = SFTTrainer(
    #     model=model,
    #     args=sft_config,
    #     train_dataset=dataset["train"],
    #     eval_dataset=dataset["valid"],
    #     tokenizer=tokenizer,
    # )
    # 
    # print("starting training...........")
    # 
    # 
    # start_time_train = time.time()
    # trainer.train()
    # end_time_train = time.time()
    # elapsed_time_train = end_time_train - start_time_train
    # 
    # print('training completed....')
    # 
    # trainer.save_model(OUTPUT_DIR)
    # 
    # print('model saved .........')
    
    test_prompts = [format_test(x, "zero") for x in dataset['test']]
    
    print(test_prompts[0])
    
    model.eval()
    
    batch_size=10
    results=[]
    start_time_test = time.time()
    with torch.no_grad():
        # for i in range(0, len(test_prompts), batch_size):
        for prompt in test_prompts:
                inputs = tokenizer(prompt, return_tensors="pt").to(device='cuda')
                # outputs = model.pipeline(inputs.input_ids)
                # batch = test_prompts[i:i + batch_size]
                # inputs = tokenizer(batch, return_tensors="pt", truncation=True,padding=True).to(device='cuda')
                outputs = model.generate(
                    inputs.input_ids, 
                    max_length=256,  # Maximum length of the generated text
                    max_new_tokens= 1,
                    num_return_sequences=1,  # Number of sequences to generate
                    no_repeat_ngram_size=2,  # Avoid repeating phrases
                    temperature=0.01,  # Controls randomness; lower is less random
                    top_k=50,  # Top-k sampling
                )
                prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
                # prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True)
                # prediction = tokenizer.decode(outputs[:, inputs.shape[1]:])
                # results.extend([x.strip() for x in prediction])
                results.append(prediction.strip())

    end_time_test = time.time()
    elapsed_time_test = end_time_test - start_time_test
    
    print("testing completed........")
    
    predictions = [x.split(" ")[-1].strip() for x in results]
    # predictions = [x.split("\n")[-1].strip() for x in results]
    
    predictions = [1 if label == "Yes" else 0 if label == "No" else 2 for label in predictions]
    # print(predictions)
    labels = [1 if label == "Yes" else 0 for label in dataset['test']['answer']]
    # print(labels)
    print(dataset_folder.split("\\"))
    
    try:
        my_mets = calc_mets_my(predictions, labels)
        print(my_mets)
        
    except Exception as e:
        print(e)
        print('my calc failed')
        my_mets = 'my calc failed'
    
    try:
        bin_mets = calculate_metrics(predictions, labels, 'binary')
        print(bin_mets)
    except Exception as e:
        print(e)
        print('binary failed')
        bin_mets = 'binary failed'
        
    try:
        micro_mets = calculate_metrics(predictions, labels, 'micro')
        print(micro_mets)
    except Exception as e:
        print(e)
        print('micro failed')
        micro_mets = 'micro failed'
        
    try:
        macro_mets = calculate_metrics(predictions, labels, 'macro')
        print(macro_mets)
    except Exception as e:
        print(e)
        print('macro failed')
        macro_mets = 'macro failed'
        
    with open("zero_plm_results.txt", "a", encoding='utf-8') as f:
        f.write(str(dataset_output_path_1))
        f.write(str(dataset_output_path_2))
        f.write('\n')
        f.write(str(dataset['train'][0]['text']))
        f.write('\n')
        f.write(str(results[0]))
        f.write('\n')
        f.write('\n')
        f.write(str(my_mets))
        f.write('\n')
        f.write(str(bin_mets))
        f.write('\n')
        f.write(str(micro_mets))
        f.write('\n')
        f.write(str(macro_mets))
        f.write('\n')
        # f.write(str(elapsed_time_train))
        f.write('\n')
        f.write(str(elapsed_time_test))
        f.write('\n')
        f.write('\n')
        f.write('********************************')
        f.write('\n')
        f.write('\n')
        
    
    del model  # Delete the model instance
    del tokenizer
    gc.collect()
    torch.cuda.empty_cache()
    


['datasets', 'my_data_plm', 'auck', '']
successfully loaded dataset.......


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

loaded model........
loaded tokenizer........
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Do the two place descriptions refer to the same real-world place? Answer with 'Yes' if they do and 'No' if they do not.<|eot_id|><|start_header_id|>user<|end_header_id|>

Place1: 'COL name VAL Gusto at the Grand COL type VAL restaurant COL latitude VAL -36.8493632 COL longitude VAL 174.762685 '
Place2: 'COL name VAL Federal Delicatessen COL type VAL restaurant COL latitude VAL -36.8489282 COL longitude VAL 174.7624718 '<|eot_id|><|start_header_id|>assistant<|end_header_id|>

No<|eot_id|>
Do the two place descriptions refer to the same real-world place? Answer with 'Yes' if they do and 'No' if they do not.
Place 1: 'COL name VAL Tautini COL type VAL farmstead COL latitude VAL -40.18825 COL longitude VAL 176.14021 '
Place 2: 'COL name VAL Waikoukou Stream COL type VAL Stream COL latitude VAL -40.08742665596005 COL longitude VAL 176.28878276483226 '

Answer: 
testing completed.......

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

...

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

successfully loaded dataset.......


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

loaded model........
loaded tokenizer........
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Do the two place descriptions refer to the same real-world place? Answer with 'Yes' if they do and 'No' if they do not.<|eot_id|><|start_header_id|>user<|end_header_id|>

Place1: 'COL name VAL Gross Chärpf COL type VAL mountain COL latitude VAL 46.91674 COL longitude VAL 9.09322 '
Place2: 'COL name VAL Chalchstöckli COL type VAL Gipfel COL latitude VAL 46.903126460802675 COL longitude VAL 9.06691611355023 '<|eot_id|><|start_header_id|>assistant<|end_header_id|>

No<|eot_id|>
Do the two place descriptions refer to the same real-world place? Answer with 'Yes' if they do and 'No' if they do not.
Place 1: 'COL name VAL Piz Mezdi COL type VAL peak COL latitude VAL 46.47009 COL longitude VAL 9.86314 '
Place 2: 'COL name VAL La Crasta COL type VAL Gipfel COL latitude VAL 46.49094755900441 COL longitude VAL 9.875518177680942 '

Answer: 
testing completed........
['datasets', 'acheson_plm'

In [18]:
dataset_folder_path = ['datasets\\gtminer_plm\\mel\\', 'datasets\gtminer_plm\\sea\\', 'datasets\gtminer_plm\\sin\\', 'datasets\\gtminer_plm\\tor\\']

In [19]:
logging.set_verbosity_error()


for dataset_folder in dataset_folder_path:
 
    warnings.filterwarnings("ignore")
    
    print(dataset_folder.split("\\"))
    dataset_output_path_1, dataset_output_path_2 = dataset_folder.split("\\")[-3], dataset_folder.split("\\")[-2]
        
    dataset = load_dataset(
        "json",
        data_files={"train": dataset_folder+"train.json", "valid": dataset_folder+"valid.json", "test": dataset_folder+"test.json"},
    )
    print("successfully loaded dataset.......")
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
        config=model_config,
        quantization_config=bnb_config,
        device_map='auto',
        token=True
    )
    
    print("loaded model........")
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        MODEL_NAME,
        token=True
    )
    
    print("loaded tokenizer........")
    PAD_TOKEN = tokenizer.eos_token
    tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
    tokenizer.padding_side = "right"
    
    
    # model = prepare_model_for_kbit_training(model)
    # model = get_peft_model(model, lora_config)
    
    print(dataset['train'][0]['text'])
    
    
    
    # OUTPUT_DIR = "experiments\\"+ dataset_output_path_1 +"\\"+ dataset_output_path_2+"\\"
    # 
    # os.makedirs(os.path.dirname(OUTPUT_DIR), exist_ok=True)
    # 
    # sft_config = SFTConfig(
    #     output_dir=OUTPUT_DIR,
    #     dataset_text_field="text",
    #     max_seq_length=300,
    #     num_train_epochs=2,
    #     per_device_train_batch_size=6,
    #     per_device_eval_batch_size=4,
    #     gradient_accumulation_steps=4,
    #     # optim="paged_adamw_8bit",
    #     # eval_strategy="steps",
    #     # eval_steps=0,
    #     #save_steps=0.2,
    #     logging_steps=10,
    #     learning_rate=1e-4,
    #     # fp16=True,  # or 
    #     bf16=True,
    #     save_strategy="steps",
    #     warmup_ratio=0.1,
    #     save_total_limit=0,
    #     lr_scheduler_type="constant",
    #     # report_to="tensorboard",
    #     save_safetensors=True,
    #     dataset_kwargs={
    #         "add_special_tokens": False,  
    #         "append_concat_token": False, 
    #     },
    # )
    # 
    # trainer = SFTTrainer(
    #     model=model,
    #     args=sft_config,
    #     train_dataset=dataset["train"],
    #     eval_dataset=dataset["valid"],
    #     tokenizer=tokenizer,
    # )
    # 
    # print("starting training...........")
    # 
    # 
    # 
    # start_time_train = time.time()
    # trainer.train()
    # end_time_train = time.time()
    # elapsed_time_train = end_time_train - start_time_train
    # 
    # print('training completed....')
    # 
    # trainer.save_model(OUTPUT_DIR)
    # 
    # print('model saved .........')
    
    # if dataset_output_path_1 =="gtminer":
    test_prompts = [format_test_gtminer(x, "zero") for x in dataset['test']]
    # elif dataset_output_path_1 =="gtminer3":
    #     test_prompts = [format_test_gtminer3(x) for x in dataset['test']]
    
    results=[]
    start_time_test = time.time()  
    
    model.eval()
    with torch.no_grad():
        for prompt in test_prompts:
                inputs = tokenizer(prompt, return_tensors="pt").to(device='cuda')
                # outputs = model.pipeline(inputs.input_ids)
                outputs = model.generate(
                    inputs.input_ids, 
                    max_length=300,  # Maximum length of the generated text
                    max_new_tokens= 2,
                    num_return_sequences=1,  # Number of sequences to generate
                    no_repeat_ngram_size=2,  # Avoid repeating phrases
                    temperature=0.01,  # Controls randomness; lower is less random
                    top_k=50,  # Top-k sampling
                )
                prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
                # prediction = tokenizer.decode(outputs[:, inputs.shape[1]:])
                results.append(prediction.strip())
    
    end_time_test = time.time()
    elapsed_time_test = end_time_test - start_time_test
    
    print("testing completed........")
    # print(results)
    predictions = [x.split(":")[-1].strip() for x in results]
    # predictions = [x.split("\n")[-1].strip() for x in results]
    
    predictions = [1 if label in ["same_as", "same", "same-as"] else 2 if label in ["part_of", "part-of", "partof"] else 3 if label in ["serves", "served"] else 0 if label in ["unknown"] else 4 for label in predictions]
    # print(predictions)
    labels = [1 if label == "same_as" else 2 if label == "part_of" else 3 if label == "serves" else 0 if label == "unknown" else 5 for label in dataset['test']['answer']]
    # print(labels)
    print(dataset_folder.split("\\"))
    
    try:
        my_mets = calculate_metrics2(predictions, labels)
        print(my_mets)
        
    except Exception as e:
        print(e)
        print('my calc failed')
        my_mets = 'my calc failed'
    
    try:
        bin_mets = calculate_metrics(predictions, labels, 'binary')
        print(bin_mets)
    except Exception as e:
        print(e)
        print('binary failed')
        bin_mets = 'binary failed'
        
    try:
        micro_mets = calculate_metrics(predictions, labels, 'micro')
        print(micro_mets)
    except Exception as e:
        print(e)
        print('micro failed')
        micro_mets = 'micro failed'
        
    try:
        macro_mets = calculate_metrics(predictions, labels, 'macro')
        print(macro_mets)
    except Exception as e:
        print(e)
        print('macro failed')
        macro_mets = 'macro failed'
        
    with open("zero_gtminer_plm_results.txt", "a", encoding='utf-8') as f:
        f.write(str(dataset_output_path_1))
        f.write(str(dataset_output_path_2))
        f.write('\n')
        f.write(str(dataset['train'][0]['text']))
        f.write('\n')
        f.write(str(results[0]))
        f.write('\n')
        f.write('\n')
        f.write(str(my_mets))
        f.write('\n')
        f.write(str(bin_mets))
        f.write('\n')
        f.write(str(micro_mets))
        f.write('\n')
        f.write(str(macro_mets))
        f.write('\n')
        # f.write(str(elapsed_time_train))
        f.write('\n')
        f.write(str(elapsed_time_test))
        f.write('\n')
        f.write('\n')
        f.write('********************************')
        f.write('\n')
        f.write('\n')
    
    del model  # Delete the model instance
    del tokenizer
    gc.collect()
    torch.cuda.empty_cache()


['datasets', 'gtminer_plm', 'mel', '']


Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

successfully loaded dataset.......


...

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

successfully loaded dataset.......


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

loaded model........
loaded tokenizer........
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Two place descriptions are provided. Answer with 'same_as' if the first place is the same as the second place. Answer with 'part_of' if the first place is a part of the second place and is located inside the second place. Answer with 'serves' if the first place provides a service to the second place in terms of human mobility, assistance, etc. Answer with 'unknown' if the two places show none of these relations.<|eot_id|><|start_header_id|>user<|end_header_id|>

Place 1: 'COL name VAL First Copy COL type VAL copyshop COL address VAL 61 College Street COL latitude VAL 43.6605215 COL longitude VAL -79.386228 '
Place 2: 'COL name VAL Medical Sciences Building COL type VAL Colleges & Universities; Education COL address VAL 1 King's College Cir M5S 1A8 COL latitude VAL 43.6607748 COL longitude VAL -79.3933729 '
Answer only with: same_as, part_of, serves, unknown<|eot_id|><|start_header

In [ ]:
MODEL_NAME = 'meta-llama/Meta-Llama-3-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained('experiments/my_data/auck')
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, 'experiments/my_data/auck')
model = model.merge_and_unload()